#Paso3. Difusión Lineal Unidimensional

$$
\frac{\partial u}{\partial t} = v\frac{\partial^2u}{\partial x^2}
$$

Esta es la ecuación del calor unidimensional. Se le llama de difusión porque a partir de un profile inicial la propiedad evaluada se difunde con el paso del tiempo, hasta alcanzar una distribución homogenea en el espacio.

En esta ecuación nos enfrentamos a un término en derivadas parciales de segundo órden. Para resolverla numéricamente podemos combinar las aproximaciones en diferencias finitas al rededor de un punto usando los esquemas FD y BD para la primera derivada. Veamos las expansiones de Taylor

$$
u(i+1) = u(i) + \Delta \frac{\partial u(i)}{\partial x} + \frac{\Delta x^2}{2!} \frac{\partial^2u(i)}{\partial x^2} + \frac{\Delta x^3}{3!} \frac{\partial^3u(i)}{\partial x^3} + ...
$$
$$
u(i-1) = u(i) - \Delta \frac{\partial u(i)}{\partial x} + \frac{\Delta x^2}{2!} \frac{\partial^2u(i)}{\partial x^2} - \frac{\Delta x^3}{3!} \frac{\partial^3u(i)}{\partial x^3} + ...
$$

Sumando las ecuaciones anteriores podemos resolver para $\frac{\partial^2u(i)}{\partial x^2}$ y obtener una ecuación de segundo órden

$$
u(i+1) + u(i-1) \approx 2u(i) + \Delta x^2 \frac{\partial^2 u(i)}{\partial x^2}
$$

$$
\frac{\partial^2 u(i)}{\partial x^2} \approx \frac{u(i+1) - 2u(i) + u(i-1)}{\Delta x^2}
$$

Con esta expresión podemos obtener la aproximación numérica de la segunda derivada en términos de datos que conocemos de las condiciones iniciales y un delta que podemos establecer nosotros. 

Debido a la naturaleza física isotrópica de la difusión este problema se analisa numéricamente con el esquema FD-CS (Forward in time, Centered in space). Si simplificamos la ecuación asumiendo que la $\textit{difusividad térmica}$, $"v"$ es constante, la discretización de la ecuación de difusión es:

$$
\frac{u(i)_{n+1} - u(i)_n}{\Delta t} = v\frac{u(i+1)_n - 2u(i)_n + u(i-1)_n}{\Delta x^2}
$$

Resolviendo para el término desconocido $u(i)_{n+1}$:

$$
u(i)_{n+1} = u(i)_n + v\frac{\Delta t}{\Delta x^2} \Big[ u(i+1)_n - 2u(i)_n + u(i-1)_n \Big]
$$

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np;

# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 90;

# The time step size 
dt = 0.01;

# Thermal diffusivity
nu = 0.1;

# spatial partitions
x_partitions = 100;

data = np.linspace(-5.0, 5.0, num=x_partitions, retstep=True);

x  = data[0];
dx = data[1];

# Set initial profile
miu = 0;
sigmaSqrt = 0.2;

u = (1.0 / np.sqrt(2*np.pi*sigmaSqrt)) * np.exp( -(x-miu)**2/(2*sigmaSqrt) );

fig = plt.figure();
fig.suptitle(r'$\frac{\partial u}{\partial t} = v\frac{\partial^2u}{\partial x^2}$', fontsize=20, fontweight='bold');
ax = plt.axes(xlim=(-5.0, 5.0), ylim=(-0.25, 1.0));
profile, = ax.plot([], [], lw=2);

def init():
    profile.set_data([], []);
    return profile,

def update(n): 
    
    if(n == 0):
        profile.set_data(x, u);
        return profile,        
    
    #store the actual state of the array
    u_n = u;
    
    #update array
    u[1:] = u_n[1:] + ((nu*dt)/(dx**2)) * (np.roll(u_n, -1)[1:] - 2*u_n[1:] + np.roll(u_n, 1)[1:]);
                    
    profile.set_data(x, u);

    return profile,

# The following lines works only on a machine with ffmpeg configured
anim = animation.FuncAnimation(fig, update, init_func=init, frames=t_seconds, blit=True);
anim.save('1D_Diffusion_01.mp4', fps=30, writer="ffmpeg", codec="libx264");
# plt.show(fig);
plt.close(fig);

In [2]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("1D_Diffusion_01.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)